In [4]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import os
family_group = "7.Papilionidae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")) and not(file.endswith("_old_backup.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
#     print(coordinate_info_dictionary)
#     print(coordinate_detail_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Battus_philenor
Exon_1 Match
[['Papilio_xuthus', ['18078601', '18078767']], ['Papilio_machaon', ['18078601', '18078767']], ['Danaus_plexippus', ['18078601', '18078767']]]
Exon_2 Match
[['Papilio_xuthus', ['18082619', '18082767']], ['Papilio_machaon', ['18082619', '18082767']], ['Danaus_plexippus', ['18082619', '18082767']]]
Exon_3 Match
[['Papilio_xuthus', ['18083566', '18083747']], ['Papilio_machaon', ['18083566', '18083747']], ['Danaus_plexippus', ['18083566', '18083747']]]
Exon_4 Match
[['Papilio_xuthus', ['18084072', '18084186']], ['Papilio_machaon', ['18084072', '18084186']], ['Danaus_plexippus', ['18084072', '18084186']]]
Exon_5 Match
[['Papilio_xuthus', ['18091708', '18091803']], ['Papilio_machaon', ['18091708', '18091803']], ['Danaus_plexippus', ['18091708', '18091803']]]
Exon_6 mismatch
[['Papilio_xuthus', ['18092311', '18092374']], ['Papilio_machaon', ['18092311', '18092374']], ['Danaus_plexippus', ['18086796', '18087039']]]
Proceed?y
Papilio_xuthus Exon_6 168_frame0 0.366 
T

Exon_1 Match
[['Papilio_xuthus', ['10828922', '10829085']], ['Papilio_machaon', ['10828922', '10829085']], ['Danaus_plexippus', ['10828922', '10829085']]]
Exon_2 Match
[['Papilio_xuthus', ['10824922', '10825070']], ['Papilio_machaon', ['10824922', '10825070']], ['Danaus_plexippus', ['10824922', '10825070']]]
Exon_3 Match
[['Papilio_xuthus', ['10823551', '10823732']], ['Papilio_machaon', ['10823551', '10823732']], ['Danaus_plexippus', ['10823551', '10823732']]]
Exon_4 Match
[['Papilio_xuthus', ['10823343', '10823460']], ['Papilio_machaon', ['10823343', '10823460']], ['Danaus_plexippus', ['10823343', '10823460']]]
Exon_5 Match
[['Papilio_xuthus', ['10820367', '10820462']], ['Papilio_machaon', ['10820367', '10820462']], ['Danaus_plexippus', ['10820367', '10820462']]]
Exon_6 Match
[['Papilio_xuthus', ['10818968', '10819031']], ['Papilio_machaon', ['10818968', '10819031']], ['Danaus_plexippus', ['10818968', '10819031']]]
Exon_7 Match
[['Papilio_xuthus', ['10814385', '10814623']], ['Papilio_

Proceed?y
Papilio_xuthus Exon_11 66_frame2 0.673 
LKSMALPRAHFYGPCHGNMVMFPQTHKGPGTTLVPQHGIGAQYLEPAPYVGAVGVPGVLSVPLQTLPVIVSSDQAQIQ
Papilio_machaon Exon_11 66_frame2 0.747 
LKSMALPRAHFYGPCHGNMVMFPQTHKGPGTTLVPQHGIGAQYLEPAPYVGAVGVPGVLSVPLQTLPVIVSSDQAQIQ
Danaus_plexippus Exon_11 66_frame2 0.605 
THKGPGTTLVPQHGIGAQYLEPAPYVGAVGVPGVLSVPLQTLPVIVSSDQAQIQ
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_xuthus_ PKPLA-QKQNDYGPCHGNMMMFSQTHKVLEPTLVPQYGIGAQYLEPAPYVGAVGVPGVLP
set66_frame2    LKSMALPRAHFYGPCHGNMVMFPQTHKGPGTTLVPQHGIGAQYLEPAPYVGAVGVPGVLS
                 *.:*  : : ********:**.****   .*****:**********************.

Papilio_xuthus_ VSLPPLPVIVSPDQTQMQ---------------
set66_frame2    VPLQTLPVIVSSDQAQIQVSFAMTGISHYCKFF
                *.* .******.**:*:*               

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon QKPVARTRNNC-GPCHGNMMMFSQTHKSLEPMLAPQYGIGAQYLEPASYVGAVGVPGVLP
set66_frame2    LKSMALPRAHFYGPCHGNMVMFPQTHKGPGTTLVPQHGIGAQYLEPAPYVGAV

Choose Species 1 - 31
Exon_13 Match
[['Papilio_xuthus', ['119307', '119462']], ['Papilio_machaon', ['119307', '119462']], ['Danaus_plexippus', ['119307', '119462']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Ornithoptera_alexandrae,JARDVY010000134.1,98947,99113,0,N,Papilio_xuthus_XM_013306791.1_Frame_0_rightoh_2_query_Exon_1,0,0,0
Ornithoptera_alexandrae,JARDVY010000134.1,102965,103113,0,N,Papilio_xuthus_XM_013306791.1_Frame_1_rightoh_1_query_Exon_2,1,49,49,Y,N
Ornithoptera_alexandrae,JARDVY010000134.1,103324,103505,0,N,Papilio_xuthus_XM_013306791.1_Frame_2_rightoh_0_query_Exon_3,1,60,60,Y,N
Ornithoptera_alexandrae,JARDVY010000134.1,103711,103828,0,N,Papilio_xuthus_XM_013306791.1_Frame_0_rightoh_1_query_Exon_4,1,39,39,Y,N
Ornithoptera_alexandrae,JARDVY010000134.1,107142,107237,0,N,Papilio_xuthus_XM_013306791.1_Frame_2_rightoh_1_query_Exon_5,1,31,31,Y,N
Ornithoptera_alexandrae,JARDVY010000134.1,107476,107536,0,N,Papilio_xuthus_XM_013306791.1_F

Exon_1 Match
[['Papilio_xuthus', ['5968115', '5968281']], ['Papilio_machaon', ['5968115', '5968281']], ['Danaus_plexippus', ['5968115', '5968281']]]
Exon_2 Match
[['Papilio_xuthus', ['5964115', '5964263']], ['Papilio_machaon', ['5964115', '5964263']], ['Danaus_plexippus', ['5964115', '5964263']]]
Exon_3 Match
[['Papilio_xuthus', ['5963854', '5964035']], ['Papilio_machaon', ['5963854', '5964035']], ['Danaus_plexippus', ['5963854', '5964035']]]
Exon_4 Match
[['Papilio_xuthus', ['5963623', '5963740']], ['Papilio_machaon', ['5963623', '5963740']], ['Danaus_plexippus', ['5963623', '5963740']]]
Exon_5 Match
[['Papilio_xuthus', ['5956840', '5956935']], ['Papilio_machaon', ['5956840', '5956935']], ['Danaus_plexippus', ['5956840', '5956935']]]
Exon_6 Match
[['Papilio_xuthus', ['5954961', '5955024']], ['Papilio_machaon', ['5954961', '5955024']], ['Danaus_plexippus', ['5954961', '5955024']]]
Exon_7 Match
[['Papilio_xuthus', ['5952179', '5952417']], ['Papilio_machaon', ['5952179', '5952417']], ['D

...

Choose Species 1 - 31
Exon_12 mismatch
[['Papilio_xuthus', ['36857384', '36857584']], ['Papilio_machaon', ['36857384', '36857584']], ['Danaus_plexippus', ['36857462', '36857584']]]
Proceed?y
Papilio_xuthus Exon_12 652_frame2 0.041 
SQISCRSRRDHVPFVVQYDASSRRRQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGILQPLIN
Papilio_machaon Exon_12 652_frame2 0.041 
SQISCRSRRDHVPFVVQYDASSRRRQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGILQPLIN
Danaus_plexippus Exon_12 652_frame2 0.179 
EQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGILQPLIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_xuthus_ ----------------------------------------------------RQEQLQRK
set652_frame2   QNFNRVLFVRTTFFLRIGWIITNTIKKKSQISCRSRRDHVPFVVQYDASSRRRQEQLQRK
                                                                    ********

Papilio_xuthus_ HEELQQMIVRQQEELRQVKEQLMLARLGILQPLVN
set652_frame2   HEELQQMIVRQQEELRQVKEQLLLARLGILQPLIN
                **********************:**********:*

CLUSTAL format alignment by MAFFT L-INS-i (v

Choose Species 1 - 33
Exon_13 Match
[['Papilio_xuthus', ['348438', '348581']], ['Papilio_machaon', ['348438', '348581']], ['Danaus_plexippus', ['348438', '348581']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Sericinus_montela,JAGSMR010000009.1,320731,320897,0,N,Papilio_xuthus_XM_013306791.1_Frame_0_rightoh_2_query_Exon_1,0,0,0
Sericinus_montela,JAGSMR010000009.1,324749,324897,0,N,Papilio_xuthus_XM_013306791.1_Frame_1_rightoh_1_query_Exon_2,1,49,49,Y,N
Sericinus_montela,JAGSMR010000009.1,325200,325381,0,N,Papilio_xuthus_XM_013306791.1_Frame_2_rightoh_0_query_Exon_3,1,60,60,Y,N
Sericinus_montela,JAGSMR010000009.1,325840,325957,0,N,Papilio_xuthus_XM_013306791.1_Frame_0_rightoh_1_query_Exon_4,1,39,39,Y,N
Sericinus_montela,JAGSMR010000009.1,329145,329240,0,N,Papilio_xuthus_XM_013306791.1_Frame_2_rightoh_1_query_Exon_5,1,31,31,Y,N
Sericinus_montela,JAGSMR010000009.1,330420,330483,0,N,Papilio_xuthus_XM_013306791.1_Frame_2_rightoh_2_query_Exon_6,1,20

Exon_1 Match
[['Papilio_xuthus', ['1153388', '1153554']], ['Papilio_machaon', ['1153388', '1153554']], ['Danaus_plexippus', ['1153388', '1153554']]]
Exon_2 Match
[['Papilio_xuthus', ['1149388', '1149536']], ['Papilio_machaon', ['1149388', '1149536']], ['Danaus_plexippus', ['1149388', '1149536']]]
Exon_3 Match
[['Papilio_xuthus', ['1148997', '1149178']], ['Papilio_machaon', ['1148997', '1149178']], ['Danaus_plexippus', ['1148997', '1149178']]]
Exon_4 Match
[['Papilio_xuthus', ['1148677', '1148794']], ['Papilio_machaon', ['1148677', '1148794']], ['Danaus_plexippus', ['1148677', '1148794']]]
Exon_5 Match
[['Papilio_xuthus', ['1145484', '1145579']], ['Papilio_machaon', ['1145484', '1145579']], ['Danaus_plexippus', ['1145484', '1145579']]]
Exon_6 mismatch
[['Papilio_xuthus', ['1145195', '1145255']], ['Papilio_machaon', ['1145195', '1145255']], ['Danaus_plexippus', ['1143820', '1143943']]]
Proceed?y
Papilio_xuthus Exon_6 78_frame1 0.856 
TNMEHMNADEPSGYQDRDS
Papilio_machaon Exon_6 78_frame1 0

Proceed?y
Papilio_xuthus Exon_6 74_frame2 0.856 
TNMEHMNADEPSGYQDRDS
Papilio_machaon Exon_6 74_frame2 0.856 
TNMEHMNADEPSGYQDRDS
Danaus_plexippus Exon_6 176_frame1 1.426 
GHTPIKSE
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_xuthus_ ---TNMEHNEADEMSGYQQEHES
set74_frame2    FSGTNMEHMNADEPSGYQDRDSR
                   ***** :*** ****:... 

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon ---TNMEHNEADEMSGYQQEHES
set74_frame2    FSGTNMEHMNADEPSGYQDRDSR
                   ***** :*** ****:... 

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu --KNLESNENGHHSYQDEHES
set176_frame1   LEKSVQADLDGHTPIKSEM-S
                  *.:::: :** . :.*  *

Choose Species 1 - 31
Exon_7 Match
[['Papilio_xuthus', ['1168682', '1168920']], ['Papilio_machaon', ['1168682', '1168920']], ['Danaus_plexippus', ['1168682', '1168920']]]
Exon_8 Match
[['Papilio_xuthus', ['1165681', '1165828']], ['Papilio_machaon', ['1165681', '1165828']], ['Danaus_pl

NotADirectoryError: [Errno 20] Not a directory: '/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/7.Papilionidae/1.Blast_result/names'